<a href="https://colab.research.google.com/github/DGkurnia/discuss1/blob/main/airquality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** [Derfansyah Guswiranata Kurnia]
- **Email:** [weerakurnia@gmail.com]
- **ID Dicoding:** [dgkurnia]

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1 : Bagaimana kondisi suhu di dua belas kota dari tahun 2013 samapai 2017 ?
- Pertanyaan 2 : Bagaimana pemeriksaan ukuran partikuler (untuk PM 2.5 dan PM 10) di dua belas kota dari tahun 2013 sampai 2017 ?.
- Pertanyaan 3 : Bagaimana pemeriksaan kadar sulfur dioksida (SO2), kadar nitrogen monoksida (NO) dan karbon monoksida (CO) di dua belas kota dari tahun 2013 sampai 2017 ?.
- Pertanyaan 4 : Bagaimana pemeriksaan kondisi gas ozone (O3) dari tahun 2013 sampai 2017 ?.

In [50]:
from google.colab import drive
drive.mount('/content/drive') #dasar instalasi file dari Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Impor Semua Paket/Library yang Digunakan
Kumpulan perpustakaan Python yang dipakai untuk proyek:


In [51]:
#persiapan perpustakaan dan file untuk pembersihan
import numpy as np
import pandas as pd #dasar pengelolaan data
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime # dasar pengelolaan data berbasis waktu
from scipy.interpolate import interp1d #interpolasi linear

## Data Wrangling
Pada tahap ini, jumlah dataframe yang akan dianalisis adalah 12 unit

In [52]:

#persiapan insersi data
dfaotizhongxin = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/aotizhongxin.csv')
dfchangping = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/changping.csv')
dfdingling = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/dingling.csv')
dfdongsi = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/dongsi.csv')
dfguanyuan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/guanyuan.csv')
dfgucheng = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/gucheng.csv')
dfhuairou = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/huairo.csv')
dfnonzhanguan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/nonzhanguan.csv')
dfshunyi = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/shunyi.csv')
dftiantan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/tiantan.csv')
dfwanliu = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/wanliu.csv')
dfwanshouxigong = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/wanshouxigong.csv')
"""
Kota awal adalah Aotizhungxin dan kota akhir adalah Wanshouxigong
"""

'\nKota awal adalah Aotizhungxin dan kota akhir adalah Wanshouxigong\n'

### Tahap Pengumpulan (Gathering Data)
#### rincian dari pengumpulan data : ada 12 dataset kota yang terdiri dari
* Nomor
* Tahun
* Bulan
* Jam
* kadar partikulat (PM 2.5 dan 10)
* kadar senyawa SO2, NO, CO, dan O3 (dalam mikrogram/ m^3)
* tempratur (dalam celsius)
* Nilai tekanan (dalam hektoPascal)
* arah angin dan kecepatan angin
-- Nilai Tahun, bulan dan jam bisa diubah menjadi tanggal.


### Assessing Data
#### Hal yang diperhatikan dalam pemeriksaan kadar udara
- Total data duplikat
-- Aotizhongxin : tidak ada
-- Changpin : Tidak ada
- Jumlah data hilang di masing-masing unit
-- Kota Aotizhingxin : 925 data di PM2.5, 718 di PM 10. Tiga kolom memiliki data kosong di atas 500
-- Kota Changpin : banyak data hilang berada di pemeriksaan empat gas tersebut
--- Kota lain memiliki kecendrungan untuk memiliki data kosong di kondisi tersebut
#### Hal unik lain
Forrmat tanggal yang salah (awalnya tahun, bulan, hari, dan jam) yang dipisah

In [53]:
"""
pemeriksaan data dari data kurang lengkap untuk enam kota pertama :
(kota Aotizhongxin sampai Huairo)
"""

#dfaotizhongxin.info()
#dfaotizhongxin.isnull().sum()
#dfaotizhongxin.duplicated().sum() # kota Aotizhongxin
#dfchangping.info()
#dfchangping.isnull().sum()
#dfchangping.duplicated().sum() # Changping
#dfdongsi.isnull().sum()
#dfdongsi.duplicated().sum()
#dfdingling.info()
#dfdingling.isnull().sum()
#dfdingling.duplicated().sum() #Dingling


'\npemeriksaan data dari data kurang lengkap untuk enam kota pertama :\n(kota Aotizhongxin sampai Huairo)\n'

In [54]:
"""
pemeriksaan data dari data kurang lengkap untuk enam kota terakhir :
(kota Nonzhanguan sampai Wanshouxigong)
"""

#dfnonzhanguan.info()
#dfnonzhanguan.isnull().sum()
#dfnonzhanguan.duplicated().sum() #Nonzhanguan
#dfwanshouxigong.info()


'\npemeriksaan data dari data kurang lengkap untuk enam kota terakhir :\n(kota Nonzhanguan sampai Wanshouxigong)\n'

### Cleaning Data
#### Pengantar : Data ini bisa dimanfaatkan untuk pengaturan sehingga proses yang sesuai untuk pembersihan data adalah interpolasi kolom indikator yang berangka demi menjaga aspek kolom yang hilang. Di sisilain, pengisian data dengan modus berlaku untuk kolom non integer
Tahapan pembersihan data
(hal ini berlaku untuk 12 kota)
1.   Pengubahan format waktu dan menghapus format salah (Tahun, bulan, angka, dan jam)
2. Proses interpolasi untuk kolom hilang di indikator yang dibutuhkan
3. Pengisian nilai non numerik yang hilang




In [55]:
#pembersihan bagian pertama (pengubahan format data waktu) untuk dua belas kota
dfaotizhongxin['datetime'] = dfaotizhongxin[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfchangping['datetime'] = dfchangping[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfdingling['datetime'] = dfdingling[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfdongsi['datetime'] = dfdongsi[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfguanyuan['datetime'] = dfguanyuan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfgucheng['datetime'] = dfgucheng[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfhuairou['datetime'] = dfhuairou[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfnonzhanguan['datetime'] = dfnonzhanguan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfshunyi['datetime'] = dfshunyi[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dftiantan['datetime'] = dftiantan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfwanliu['datetime'] = dfwanliu[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfwanshouxigong['datetime'] = dfwanshouxigong[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)

#penghapusan kolom 'tahun' dan tiga indeks lain untuk 12 kota (dari Aotizhongxin sampai Wanshouxigong)

dfaotizhongxin.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfchangping.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfdingling.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfdongsi.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfguanyuan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfgucheng.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfhuairou.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfnonzhanguan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfshunyi.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dftiantan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfwanliu.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfwanshouxigong.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)

#pemeriksaan kolom untuk memeriksa jika kota sudah memiliki indikator waktu
#dfaotizhongxin.head(10) #Kota Aotizhongxin
#dfchangping.head(10)# Kota Changping
#dfdingling.head(10)# Kota Dingling
#dfdongsi.head(10)# Kota Dongsi
#dfguanyuan.head(10)# Kota Guanyuan
#dfgucheng.head(10)# Kota Gucheng
#dfhuairou.head(10)# Kota Huairou
#dfnonzhanguan.head(10) #kota Nanzhanguan
#dfshunyi.head(10) #kota sunyi
#dftiantan.head(10) #Kota Tiantan
#dfwanliu.head(10) #Kota Wanliu
#dfwanshouxigong.head(10) #Kota Wanshouxigong
#hasil ini untuk memeriksa penghapusan kolom waktu

In [56]:
"""
#bagian dua (proses interpolasi kolom)
Tahap pertama pemeriksaan data yang hilang untuk masing-masing kota dari Aotizhongxing sampai Wanshouxigong
"""
# pemeriksan total kolom kosong untuk interpolasi
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Aotizhongxing)
#dfchangping.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Changping)
#dfdingling.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Dingling)
#dfdongsi.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Dongsi)
#dfguanyuan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Guanyuan)
#dfgucheng.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Gucheng)
#dfhuairou.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Huairou)
#dfnonzhanguan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Nonzhanguan)
#dfshunyi.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Shunyi)
#dftiantan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Tiantan)
#dfwanliu.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Wanliu)
#dfwanshouxigong.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Wanshouxigong)

'\n#bagian dua (proses interpolasi kolom)\nTahap pertama pemeriksaan data yang hilang untuk masing-masing kota dari Aotizhongxing sampai Wanshouxigong\n'

##Resume Kolom Hilang Data untuk Interpolasi Linear:

*   Sebagian besar kolom hilang bisa 'diinterpolasi' untuk menjaga orisinilitas data
*   Proses ini melibatkan dua belas Kota



In [57]:
# persiapan interpolasi (Aotizhongxin sampai terakhir) di kolom PM2.5 dan PM10 sebelum pembersihan data
dfaotizhongxin['PM2.5'] = dfaotizhongxin['PM2.5'].interpolate(method='nearest')
dfaotizhongxin['PM10'] = dfaotizhongxin['PM10'].interpolate(method='nearest')
#Area Aotizhongxin
dfchangping['PM2.5'] = dfchangping['PM2.5'].interpolate(method='nearest')
dfchangping['PM10'] = dfchangping['PM10'].interpolate(method='nearest')
#Area Changping
dfdingling['PM2.5'] = dfdingling['PM2.5'].interpolate(method='nearest')
dfdingling['PM10'] = dfdingling['PM10'].interpolate(method='nearest')
#Area Dingling
dfdongsi['PM2.5'] = dfdongsi['PM2.5'].interpolate(method='nearest')
dfdongsi['PM10'] = dfdongsi['PM10'].interpolate(method='nearest')
#Area Dongsi
dfguanyuan['PM2.5'] = dfguanyuan['PM2.5'].interpolate(method='nearest')
dfguanyuan['PM10'] = dfguanyuan['PM10'].interpolate(method='nearest')
#Area Guanyuan
dfgucheng['PM2.5'] = dfgucheng['PM2.5'].interpolate(method='nearest')
dfgucheng['PM10'] = dfgucheng['PM10'].interpolate(method='nearest')
#Area Gucheng
dfhuairou['PM2.5'] = dfhuairou['PM2.5'].interpolate(method='nearest')
dfhuairou['PM10'] = dfhuairou['PM10'].interpolate(method='nearest')
#Area Huairou
dfnonzhanguan['PM2.5'] = dfnonzhanguan['PM2.5'].interpolate(method='nearest')
dfnonzhanguan['PM10'] = dfnonzhanguan['PM10'].interpolate(method='nearest')
#Area Nonzhanguan
dfshunyi['PM2.5'] = dfshunyi['PM2.5'].interpolate(method='nearest')
dfshunyi['PM10'] = dfshunyi['PM10'].interpolate(method='nearest')
#Area Shunyi
dftiantan['PM2.5'] = dftiantan['PM2.5'].interpolate(method='nearest')
dftiantan['PM10'] = dftiantan['PM10'].interpolate(method='nearest')
#Area Tiantan
dfwanliu['PM2.5'] = dfwanliu['PM2.5'].interpolate(method='nearest')
dfwanliu['PM10'] = dfwanliu['PM10'].interpolate(method='nearest')
#Area Wanliu
dfwanshouxigong['PM2.5'] = dfwanshouxigong['PM2.5'].interpolate(method='nearest')
dfwanshouxigong['PM10'] = dfwanshouxigong['PM10'].interpolate(method='nearest')
#Area Wanshouxigong

In [58]:
#pemeriksaan nilai interpolasi linear untuk dua PM
#dfaotizhongxin[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Aotizhongxing)
#dfchangping[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Changping)
#dfdingling[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Dingling)
#dfdongsi[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Dongsi)
#pemeriksaan hasil interpolasi untuk nilai dua jenis meter
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi PM (Aotizhongxing)
#dfchangping.isnull().sum() #untuk memeriksa kolom setelah interpolasi PM (Changping)

##Bagian 2 pembersihan dataset untuk pemeriksaan empat senyawa (dalam mikro meter/ meter kubik)

In [59]:
#pembersihan data SO2, NO2, CO O3 di dataset (Aotizhongxin sampai Wanshouxigong)
dfaotizhongxin[['SO2', 'NO2', 'CO', 'O3']] = dfaotizhongxin[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Aotizhongxin
dfchangping[['SO2', 'NO2', 'CO', 'O3']] = dfchangping[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Changping
dfdingling[['SO2', 'NO2', 'CO', 'O3']] = dfdingling[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Dingling
#dfdongsi[['SO2', 'NO2', 'CO', 'O3']] = dfdongsi[[['SO2', 'NO2', 'CO', 'O3']]].interpolate(method='nearest') #Kota Dongsi
dfguanyuan[['SO2', 'NO2', 'CO', 'O3']] = dfguanyuan[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Guanyuan
dfgucheng[['SO2', 'NO2', 'CO', 'O3']] = dfgucheng[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Gucheng
dfhuairou[['SO2', 'NO2', 'CO', 'O3']] = dfhuairou[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Huairou
dfnonzhanguan[['SO2', 'NO2', 'CO', 'O3']] = dfnonzhanguan[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Nonzhanguan
dfshunyi[['SO2', 'NO2', 'CO', 'O3']] = dfshunyi[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Shunyi
dfwanliu[['SO2', 'NO2', 'CO', 'O3']] = dfwanliu[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Wanliu
dfwanshouxigong[['SO2', 'NO2', 'CO', 'O3']] = dfwanshouxigong[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='nearest') #Kota Wanshouxigong

#pemeriksaan interpolasi konsentrasi gas
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi (Aotizhongxing)
dfdongsi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   No        35064 non-null  int64         
 1   PM2.5     35064 non-null  float64       
 2   PM10      35064 non-null  float64       
 3   SO2       34401 non-null  float64       
 4   NO2       33463 non-null  float64       
 5   CO        31867 non-null  float64       
 6   O3        34400 non-null  float64       
 7   TEMP      35044 non-null  float64       
 8   PRES      35044 non-null  float64       
 9   DEWP      35044 non-null  float64       
 10  RAIN      35044 non-null  float64       
 11  wd        34986 non-null  object        
 12  WSPM      35050 non-null  float64       
 13  station   35064 non-null  object        
 14  datetime  35064 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(11), int64(1), object(2)
memory usage: 4.0+ MB


###Interpolasi suhu, tekanan, dan nilai tekanan dew secara linear

In [60]:
#persiapan proses interpolasi
dfaotizhongxin[['TEMP','PRES','DEWP']] = dfaotizhongxin[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Aotizhongxin
dfchangping[['TEMP','PRES','DEWP']] = dfchangping[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Changping
dfdingling[['TEMP','PRES','DEWP']] = dfdingling[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Dingling
dfdongsi[['TEMP','PRES','DEWP']] = dfdongsi[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Dongsi
dfguanyuan[['TEMP','PRES','DEWP']] = dfguanyuan[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Guanyuan
dfgucheng[['TEMP','PRES','DEWP']] = dfgucheng[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Gucheng
dfhuairou[['TEMP','PRES','DEWP']] = dfhuairou[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Huairou
dfnonzhanguan[['TEMP','PRES','DEWP']] = dfnonzhanguan[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi[['TEMP','PRES','DEWP']] = dfshunyi[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Shunyi
dfwanliu[['TEMP','PRES','DEWP']] = dfwanliu[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Wanliu
dfwanshouxigong[['TEMP','PRES','DEWP']] = dfwanshouxigong[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Wanshouxigong


#Proses interpolasi untuk kadar hujan
dfaotizhongxin['RAIN'] = dfaotizhongxin['RAIN'].interpolate(method='linear') #Kota Aotizhongxin
dfchangping['RAIN'] = dfchangping['RAIN'].interpolate(method='linear') #Kota Changping
dfdingling['RAIN'] = dfdingling['RAIN'].interpolate(method='linear') #Kota Dingling
dfdongsi['RAIN'] = dfdongsi['RAIN'].interpolate(method='linear') #Kota Dongsi
dfguanyuan['RAIN'] = dfguanyuan['RAIN'].interpolate(method='linear') #Kota Guanyuan
dfgucheng['RAIN'] = dfgucheng['RAIN'].interpolate(method='linear') #Kota Gucheng
dfhuairou['RAIN'] = dfhuairou['RAIN'].interpolate(method='linear') #Kota Huairou
dfnonzhanguan['RAIN'] = dfnonzhanguan['RAIN'].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi['RAIN'] = dfshunyi['RAIN'].interpolate(method='linear') #Kota Shunyi
dfwanliu['RAIN'] = dfwanliu['RAIN'].interpolate(method='linear') #Kota Wanliu
dfwanshouxigong['RAIN'] = dfwanshouxigong['RAIN'].interpolate(method='linear') #Kota Wanshouxigong

#pemeriksaan hasil interpolasi
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi kadar suhu(Aotizhongxing)

##Pengisian data hilang untuk arah angin dan kecepatan


In [87]:
#persiapan pengisian data hilang di arah angin
dfaotizhongxin['wd'] = dfaotizhongxin['wd'].fillna(method='ffill') #kota aotizhongxin
dfchangping['wd'] = dfchangping['wd'].fillna(method='ffill')
dfdingling['wd'] = dfdingling['wd'].fillna(method='ffill')
dfdongsi['wd'] = dfdongsi['wd'].fillna(method='ffill')
dfguanyuan['wd'] = dfguanyuan['wd'].fillna(method='ffill')
dfgucheng['wd'] = dfgucheng['wd'].fillna(method='ffill')
dfhuairou['wd'] = dfhuairou['wd'].fillna(method='ffill')
dfnonzhanguan['wd'] = dfnonzhanguan['wd'].fillna(method='ffill')
dfshunyi['wd'] = dfshunyi['wd'].fillna(method='ffill')
dftiantan['wd'] = dftiantan['wd'].fillna(method='ffill')
dfwanliu['wd'] = dfwanliu['wd'].fillna(method='ffill')
dfwanshouxigong['wd'] = dfwanshouxigong['wd'].fillna(method='ffill')

#interpolasi linear untuk kecepatan angin
dfaotizhongxin['WSPM'] = dfaotizhongxin['WSPM'].interpolate(method='linear') #Kota Aotizhongxin
dfchangping['WSPM'] = dfchangping['WSPM'].interpolate(method='linear') #Kota Changping
dfdingling['WSPM'] = dfdingling['WSPM'].interpolate(method='linear') #Kota Dingling
dfdongsi['WSPM'] = dfdongsi['WSPM'].interpolate(method='linear') #Kota Dongsi
dfguanyuan['WSPM'] = dfguanyuan['WSPM'].interpolate(method='linear') #Kota Guanyuan
dfhuairou['WSPM'] = dfhuairou['WSPM'].interpolate(method='linear') #Kota Huairou
dfnonzhanguan['WSPM'] = dfnonzhanguan['WSPM'].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi['WSPM'] = dfshunyi['WSPM'].interpolate(method='linear') #Kota Shunyi
dftiantan['WSPM'] = dftiantan['WSPM'].interpolate(method='linear') #Kota Tiantan
dfwanliu['WSPM'] = dfwanliu['WSPM'].interpolate(method='linear') #Kota Wanliu
dfwanshouxigong['WSPM'] = dfwanshouxigong['WSPM'].interpolate(method='linear') #Kota Wanshouxigong



## Exploratory Data Analysis (EDA)

In [ ]:
#pemeriksaan data kosong (dari Aotizhongxing sampai Huatiou)
#dfaotizhongxin.info()
#persiapan pemeriksana ukuran partikulat (aotizhongxin)
# Ukuran grafik pertama
plt.figure(figsize=(19, 8))

# Grafik histograms untuk PM2.5 di semua kota
sns.histplot(dfaotizhongxin['PM2.5'], bins=30, kde=True, color='skyblue', label='PM2.5 di Aotizhongxin', alpha=0.6)
# Kota Changping
sns.histplot(dfchangping['PM2.5'], bins=30, kde=True, color='black', label='PM2.5 di Changping', alpha=0.35)
#area Dingling
sns.histplot(dfdingling['PM2.5'], bins=30, kde=True, color='pink', label='PM2.5 di Dingling', alpha=0.75)
#area Dongsi
sns.histplot(dfdongsi['PM2.5'], bins=30, kde=True, color='violet', label='PM2.5 di Dongsi', alpha=0.75)
#Area Guanyuan
sns.histplot(dfguanyuan['PM2.5'], bins=30, kde=True, color='darkviolet', label='PM2.5 di Guanyuan', alpha=0.25)
#Area Guchen
sns.histplot(dfgucheng['PM2.5'], bins=30, kde=True, color='green', label='PM2.5 di Gucheng', alpha=1.0)
#Area Huatiou
sns.histplot(dfhuairou['PM2.5'], bins=30, kde=True, color='red', label='PM2.5 di Gucheng', alpha=0.20)
#Area Nonzhanguan
sns.histplot(dfnonzhanguan['PM2.5'], bins=30, kde=True, color='orange', label='PM2.5 at Nonzhanguan', alpha=0.53)
#Area Shunyi
sns.histplot(dfshunyi['PM2.5'], bins=30, kde=True, color='teal', label='PM2.5 di Shunyi', alpha=0.45)
#Area Tiantan
sns.histplot(dftiantan['PM2.5'], bins=30, kde=True, color='white', label='PM2.5 di Tiantan', alpha=0.55)
#Area Wanliu
sns.histplot(dfwanliu['PM2.5'], bins=30, kde=True, color='grey', label='PM2.5 di Wanliu', alpha=0.95)
#Area Wanshouxigong
sns.histplot(dfwanshouxigong['PM2.5'], bins=30, kde=True, color='blue', label='PM2.5 di Wanshouxigong', alpha=0.70)


# Penambahan legenda
plt.legend(loc='upper right', fontsize='medium')

# Judul grafik
plt.title('Distribution of PM2.5 di semua kota', fontsize=20)
plt.xlabel('Ukuran partikulat (µg/m³)', fontsize=18)
plt.ylabel('Frequency', fontsize=18)

#Bagian dua untuk pemeriksaan kecundrangan di ukuran 10M
plt.figure(figsize=(19, 8))
#pemeriksan seaborn
sns.histplot(dfaotizhongxin['PM10'], bins=30, kde=True, color='orange', label='PM10 di Aotizhongxin', alpha=0.6)
#Changping
sns.histplot(dfchangping['PM10'], bins=30, kde=True, color='red', label='PM10 di Changping', alpha=0.35)
#Dingling
sns.histplot(dfdingling['PM10'], bins=30, kde=True, color='darkblue', label='PM10 di Dingling', alpha=0.75)
#Dongsi
sns.histplot(dfdongsi['PM10'], bins=30, kde=True, color='grey', label='PM10 di Dongsi', alpha=0.75)
#Guanyuan
sns.histplot(dfguanyuan['PM10'], bins=30, kde=True, color='black', label='PM10 di Guanyuan', alpha=0.25)
#Gucheng
sns.histplot(dfgucheng['PM2.5'], bins=30, kde=True, color='blue', label='PM10 di Gucheng', alpha=1.0)
#Huairou
sns.histplot(dfhuairou['PM10'], bins=30, kde=True, color='pink', label='PM10 at Huairou', alpha=0.30)
#Area Nonzhanguan
sns.histplot(dfnonzhanguan['PM10'], bins=30, kde=True, color='orange', label='PM2.5 at Nonzhanguan', alpha=0.60)
#Area Shunyi
sns.histplot(dfshunyi['PM10'], bins=30, kde=True, color='green', label='PM2.5 at Shunyi', alpha=0.65)
#Area Tiantan
sns.histplot(dftiantan['PM10'], bins=30, kde=True, color='violet', label='PM2.5 at Tiantan', alpha=0.55)
#Area Wanliu
sns.histplot(dfwanliu['PM10'], bins=30, kde=True, color='white', label='PM2.5 at Wanliu', alpha=0.95)
#Area Wanshouxigong
sns.histplot(dfwanshouxigong['PM10'], bins=30, kde=True, color='yellow', label='PM2.5 at Wanshouxigong', alpha=0.60)

# Penambahan legenda
plt.legend(loc='upper right', fontsize='medium')

# Judul grafik
plt.title('Distibusi PM10 di semua kota', fontsize=20)
plt.xlabel('Ukuran partikulat (µg/m³)', fontsize=18)
plt.ylabel('Frequency', fontsize=18)


In [ ]:
#pemeriksaan konsentrasi senyawa SO2


#Pemeriksaan senyawa NO2


**Insight:**
- xxx
- xxx

**Insight:**
- xxx
- xxx

**Insight:**
- xxx
- xxx

### Explore The insight

In [63]:
# eksplorasi ukuran partikulat



**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [64]:
#Tahap awal visualisasi untuk pertanyaan 1


### Pertanyaan 2:

In [65]:
#Tahap awal visualisasi untuk pertanyaan 1


**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2